# **SCRAPING TWITTER POSTS FOR *PlayStation* and *Xbox***

---

To collect informal opinions about PlayStation and Xbox brands we scraped one of the most used social media : Twitter.
For this purpose, we did not use the Twitter API, since historical tweets (older than one week) can not be retrieved with the free developer account, but need a payed full account.

Therefore, tweets are extracted with standard scraping techniques, using `Selenium` to simulate a browser that scrolls the Twitter web page containing the results of our advanced query, and `Beautiful Soup` to parse the web content.


ABOUT THE DATASETS
For each brand, we extracted data from 2 different temporal windows:
 - from **22 Sep 2017** to **1 Feb 2018**
 - from **22 Sep 2020** to **1 Feb 2021**

The main reason behind is that we want to retrieve almost recent opinions mostly about the most recent products:
-  **Xbox Series X** and **S**, released in November 10, 2020,
-  **Xbox One X**, released in  November 7, 2017
- **PlayStation 5**, released in November 12, 2020

and also previous consoles as Xbox One, PlayStation 4, ...



## Initialization

In [ ]:
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# open it, go to a website, and get results
wd = webdriver.Chrome('chromedriver',options=options)
wd.get("https://www.website.com")
print(wd.page_source)  # results

In [ ]:
import time
import re
import csv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

## Functions

This function creates a csv file to store scraped data

In [ ]:
def create_file(filename):
  with open(filename, 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['username', 'datetime', 'tweet_text', 'retweeted_text', 
                     'likes', 'retweets', 'comments'])

This function is used to scrape tweet information using the advanced query. Also, it avoids to store duplicates of tweets due to the scrolling of the page, taking track of the last stored tweet.

Parameters:
- `ANDterms` : they are forced to be in each tweet;
- `ORterms` : at least one between these term should be contained in the resulting tweets;
- `NOTterms` : none of these terms should be contained;
- `hashtags` : terms that are forced to be contained in the hashtag format;
- `lang` : language to filter on;
- `date_until` : end of the selected temporal window;
- `date_since` : start of the selected temporal window;
- `filename` : file in which we have to store the data.

These parameters, expecially `NOTterms` were needed to filter relevant information as much as possible, to overcome the "*dirty nature*" of Twitter (full of adv and irrelevant info).

In [ ]:
def scrape_tweets(ANDterms, ORterms, NOTterms, hashtags, lang, date_until, date_since, filename):

  ## SET URL AND BROWSER WITH SELENIUM
  browser = webdriver.Chrome('chromedriver',options=options)

  ## CONSTRUCTING THE URL USING QUERY TERMS
  base_url = u'https://twitter.com/search?q='
  for t in ANDterms:
    base_url += t + '%20'
  if len(ORterms) > 0:
    base_url += '('
  for t in ORterms:
    base_url += t
    if t == ORterms[-1]:
      base_url += ')%20'    
    else:
      base_url += '%20OR%20'
  for t in NOTterms:
    base_url += '-'+ t + '%20'
  if len(hashtags) > 0:
    base_url += '('
    for t in hashtags:
      base_url += '%23' + t
      if t == hashtags[-1]:
        base_url += ')%20'
  base_url += 'lang%3A' + lang + '%20'
  base_url += u'until%3A' + date_until + '%20' 
  base_url += u'since%3A'+date_since + '&f=live'  #from recent
  #base_url += u'since%3A'+date_since #from popular
  print(base_url)        

  browser.get(base_url)
  time.sleep(2) #wait that the page is loaded
  body = browser.find_element_by_tag_name('body')

  count = 0

  last_tweet = ""
  started = True #helps to remember if we already started to scrape tweets (False if we are still checking for overlaps)

  try:
    while True:   #scroll pages until KeyBoard interruption
      print("\n\n . . . . . . SCROLLING PAGE . . . . . .")
      body.send_keys(Keys.PAGE_DOWN)
      time.sleep(0.2) #wait that the content is loaded

      ## GET HTML CONTENT
      html_source = browser.page_source
      soup = BeautifulSoup(html_source)

      ## GET TWEETS
      for d in soup.findAll('article', attrs={'class':'css-1dbjc4n r-1loqt21 r-18u37iz r-1ny4l3l r-1udh08x r-1yt7n81 r-ry3cjt r-o7ynqc r-6416eg'}): 

        #Get tweet text and hashtags
        text_container = d.find('div', attrs={'class':'css-901oao r-18jsvk2 r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0'})
        text_children = text_container.findChildren('span')
        text = ""
        for text_tag in text_children:
          text += text_tag.text
        text = re.sub('\s+',' ',text)  #removing multiple spaces and new lines

        if (last_tweet == "") or (last_tweet != text and started == True):

          ## DON'T CONSIDER TWEETS WITH http links (just ads, irrelevant)
          links = d.find('span', attrs={'class':'css-901oao css-16my406 r-poiln3 r-hiw28u r-bcqeeo r-qvutc0'})
          if links is None:

            count += 1

            #Get username
            user_tag = d.find('span', attrs={'class':'css-901oao css-16my406 r-poiln3 r-bcqeeo r-qvutc0'})
            user = user_tag.text

            #Get date time
            date_tag = d.find('time')
            date = date_tag['datetime']

            #Get number of likes
            likes_tag = d.find('div', attrs={'data-testid':'like'})
            n_likes = (likes_tag['aria-label']).split(" ")[0]

            #Get number of retweets
            retweets_tag = d.find('div', attrs={'data-testid':'retweet'})
            n_retweets = (retweets_tag['aria-label']).split(" ")[0]

            #Get number of comments
            comments_tag = d.find('div', attrs={'data-testid':'reply'})
            n_comments = (comments_tag['aria-label']).split(" ")[0]

            #Get retwitted text
            retweetedtext_container = d.find('div', attrs={'class':'css-1dbjc4n r-1bs4hfb r-1867qdf r-rs99b7 r-1loqt21 r-dap0kf r-1ny4l3l r-1udh08x r-o7ynqc r-6416eg'})
            retweeted_text = ""
            if retweetedtext_container is not None:
              retweetedtext_children = retweetedtext_container.findChildren('span')
              for retweetedtext_tag in retweetedtext_children:
                retweeted_text += retweetedtext_tag.text
              retweeted_text = re.sub('\s+',' ',retweeted_text)  #removing multiple spaces and new lines
      
            print("\n - (", count, ") ", user, "[", date, "] >> ", text )
            if retweeted_text != "":
              print("[ RETWEETS >> ", retweeted_text, " ]")
            print(n_likes, " likes, ", n_retweets, " retweets, ", n_comments, " comments")

            ## APPEND TWEET TO CSV
            with open(filename, 'a') as f:
              writer = csv.writer(f)
              writer.writerow([user, date, text, retweeted_text, n_likes, n_retweets, n_comments])

        else:
          if last_tweet == text and started == False :
            started = True #from next tweet element I have to start scraping (no more overlaps)
    
      last_tweet = text #update last tweet with the last text of the scrolled page
      started = False

  except KeyboardInterrupt:     #interrupt the script when you see no new tweet for many ... SCROLLING PAGE ...
    print('*********** SCRAPING STOPPED! ***********')
    print("\n\nScraped  ", count, " tweets.")

This method is used for **post-filter** tweets.
Again, we noticed that the datasets still contained irrelevant information (such as "Merry Christmas #xbox #xboxseries #playstation"), so using our intuitions we inspected them, selecting tweets containing certain substrings, and took a decision about what had to be dropped and what was actually relevant.

The parameter `method` has value:
- '*view*' to just inspect tweets containing the substring (`word`)
- '*drop*' to drop all tweets containing the given substring;

In [ ]:
def view_or_drop_by_word(df,word, method):
  for index in df.index[df['tweet_text'].str.contains(word)]:
    if method == 'view':
      print(index, " >> ", df.loc[index]['tweet_text'])
      
    if method == 'drop':
      df = df.drop([index])
  if method == 'drop':
    print("Data after dropping tweets containing word", word, ": ", df['tweet_text'].count())
  return df


## Scraping *PlayStation*

### **2020-2021**

In this section we extract data from 22-09-2020 to 01-02-2021, to capture opinions regarding **PlayStation** and ps4 and ps5 products. This window considers the period before and after that ps5 was out (November 2020).

In [ ]:
## SETTING THE ADVANCED QUERY
ANDterms = ['playstation'] #all of these terms are mandatory
ORterms = ['ps3','ps4', 'ps5', 'playstation3', 'playstation4', 'playstation5'] #at least one of these terms should be included
NOTterms = ['follow', 'followme', 'check', 'giveaway', 'giveaways', 'page', 'twitch',
            'live', 'livestream', 'PS4share', 'PS5share', 'eBay', 'screenshot', 'youtube',
            'challenge', 'dm', 'shots', 'video', 'stream', 'Ad'] #none of these terms (probably add spam and irrelevant info)
hashtags = ['PlayStation'] #all these hashtags
lang = 'en'
date_until = '2021-02-02'
date_since = '2020-09-22'


## PREPARING THE FILE
filename = 'playstation_' + date_since + '_' + date_until + '_tweets.txt'
create_file(filename)

scrape_tweets(ANDterms, ORterms, NOTterms, hashtags, lang, date_until, date_since, filename)

In [ ]:
import pandas as pd
df = pd.read_csv(r'playstation_2020-09-22_2021-02-02_tweet.csv')
df.shape

(21777, 7)

We dropped duplicates due to retweeted texts:

In [ ]:
print("Data before dropping duplicate texts:\n ", df.count())
df.drop_duplicates(subset ="tweet_text",  keep = False, inplace = True)
print("Data after dropping duplicate texts:\n ", df.count())
#NOTE: some usernames are empty because they contain strange characters

Data before dropping duplicate texts:  username          25804
datetime          25854
tweet_text        25854
retweeted_text    1055 
likes             25854
retweets          25854
comments          25854
dtype: int64
Data after dropping duplicate texts:  username          21728
datetime          21777
tweet_text        21777
retweeted_text    1010 
likes             21777
retweets          21777
comments          21777
dtype: int64


We **post-filtered** tweets to remove irrelevant information.

In [ ]:
view_or_drop_by_word(df, 'Merry Christmas', 'view')
df = view_or_drop_by_word(df, 'Merry Christmas', 'drop')

view_or_drop_by_word(df, 'Happy new year', 'view')
df = view_or_drop_by_word(df, 'Happy new year', 'drop')

view_or_drop_by_word(df, 'Click', 'view')
df = view_or_drop_by_word(df, 'Click', 'drop')

view_or_drop_by_word(df, 'followers', 'view')
df = view_or_drop_by_word(df, 'followers', 'drop')

view_or_drop_by_word(df, 'contact me', 'view')
df = view_or_drop_by_word(df, 'contact me', 'drop')

view_or_drop_by_word(df, 'watch me', 'view')
df = view_or_drop_by_word(df, 'watch me', 'drop')

view_or_drop_by_word(df, 'twitchstreamers', 'view')
df = view_or_drop_by_word(df, 'twitchstreamers', 'drop')

view_or_drop_by_word(df, 'Gaming News Drop', 'view')
df = view_or_drop_by_word(df, 'Gaming News Drop', 'drop')

view_or_drop_by_word(df, 'twitchstreams', 'view')
df = view_or_drop_by_word(df, 'twitchstreams', 'drop')

view_or_drop_by_word(df, 'News Drop', 'view')
df = view_or_drop_by_word(df, 'News Drop', 'drop')

view_or_drop_by_word(df, 'Join', 'view')
df = view_or_drop_by_word(df, 'Join', 'drop')

view_or_drop_by_word(df, 'giving away', 'view')
df = view_or_drop_by_word(df, 'giving away', 'drop')

view_or_drop_by_word(df, 'link in bio', 'view')
df = view_or_drop_by_word(df, 'link in bio', 'drop')

view_or_drop_by_word(df, 'join me', 'view')
df = view_or_drop_by_word(df, 'join me', 'drop')

view_or_drop_by_word(df, 'FREE DOWNLOAD', 'view')
df = view_or_drop_by_word(df, 'FREE DOWNLOAD', 'drop')

view_or_drop_by_word(df, 'FREE', 'view')
df = view_or_drop_by_word(df, 'FREE', 'drop')


The dataset was chunked to avoid crashes during the preprocessing.

In [ ]:
GROUP_SIZE = 4000

def chunk(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

i = 0
for df_chunk in chunk(df, GROUP_SIZE):
  i += 1
  print(df_chunk['tweet_text'].count())
  file_name = 'playstation_2020-09-22_2021-02-02_tweets' + str(i) + '.csv'
  print(file_name, "saved.")
  df_chunk.to_csv(file_name, encoding='utf-8', index=False)

4000
playstation_2020-09-22_2021-02-02_tweets1.csv saved.
4000
playstation_2020-09-22_2021-02-02_tweets2.csv saved.
4000
playstation_2020-09-22_2021-02-02_tweets3.csv saved.
4000
playstation_2020-09-22_2021-02-02_tweets4.csv saved.
4000
playstation_2020-09-22_2021-02-02_tweets5.csv saved.
1474
playstation_2020-09-22_2021-02-02_tweets6.csv saved.


### **2017-2018**

We extract data from 22-09-2017 to 01-02-2018, to capture opinions regarding **PlayStation** and ps3 and ps4 products. 

In [ ]:
## SETTING THE ADVANCED QUERY
ANDterms = ['playstation'] #all of these terms are mandatory
ORterms = ['ps3','ps4', 'ps2', 'playstation2' 'playstation3', 'playstation4', 'psp'] #at least one of these terms should be included
NOTterms = ['follow', 'followme', 'check', 'giveaway', 'giveaways', 'page', 'twitch',
            'live', 'livestream', 'PS4share', 'PS5share', 'eBay', 'screenshot', 'youtube',
            'challenge', 'dm', 'shots', 'video', 'stream', 'Ad'] #none of these terms (probably add spam and irrelevant info)
hashtags = ['PlayStation'] #all these hashtags
lang = 'en'
date_until = '2018-02-02'
date_since = '2017-09-22'


## PREPARING THE FILE
filename = 'playstation_' + date_since + '_' + date_until + '_tweets.txt'
create_file(filename)

scrape_tweets(ANDterms, ORterms, NOTterms, hashtags, lang, date_until, date_since, filename)

In [ ]:
import pandas as pd
df = pd.read_csv(r'playstation_2017-09-22_2018-02-02_tweet.csv')
df.shape

(4290, 7)

In [ ]:
print("Data before dropping duplicate texts:\n ", df.count())
df.drop_duplicates(subset ="tweet_text",  keep = False, inplace = True)
print("Data after dropping duplicate texts:\n ", df.count())

Data before dropping duplicate texts:
  username          4730
datetime          4741
tweet_text        4741
retweeted_text    70  
likes             4741
retweets          4741
comments          4741
dtype: int64
Data after dropping duplicate texts:
  username          4279
datetime          4290
tweet_text        4290
retweeted_text    66  
likes             4290
retweets          4290
comments          4290
dtype: int64


**Post filtering**

In [ ]:
view_or_drop_by_word(df, 'Merry Christmas', 'view')
df = view_or_drop_by_word(df, 'Merry Christmas', 'drop')

view_or_drop_by_word(df, 'Happy new year', 'view')
df = view_or_drop_by_word(df, 'Happy new year', 'drop')

view_or_drop_by_word(df, 'followers', 'view')
df = view_or_drop_by_word(df, 'followers', 'drop')

view_or_drop_by_word(df, 'contact me', 'view')
df = view_or_drop_by_word(df, 'contact me', 'drop')

view_or_drop_by_word(df, 'Join', 'view')
df = view_or_drop_by_word(df, 'Join', 'drop')

view_or_drop_by_word(df, 'my channel', 'view')
df = view_or_drop_by_word(df, 'my channel', 'drop')

view_or_drop_by_word(df, 'youtuber', 'view')
df = view_or_drop_by_word(df, 'youtuber', 'drop')

view_or_drop_by_word(df, 'See others view', 'view')
df = view_or_drop_by_word(df, 'See others view', 'drop')

view_or_drop_by_word(df, 'How to solve', 'view')
df = view_or_drop_by_word(df, 'How to solve', 'drop')

view_or_drop_by_word(df, 'New Daily Update', 'view')
df = view_or_drop_by_word(df, 'New Daily Update', 'drop')

view_or_drop_by_word(df, 'NEW', 'view')
df = view_or_drop_by_word(df, 'NEW', 'drop')

view_or_drop_by_word(df, 'How to Solve', 'view')
df = view_or_drop_by_word(df, 'How to Solve', 'drop')

view_or_drop_by_word(df, '#YoutubeChannel #Youtuber', 'view')
df = view_or_drop_by_word(df, '#YoutubeChannel #Youtuber', 'drop')

view_or_drop_by_word(df, 'In-Stock Today', 'view')
df = view_or_drop_by_word(df, 'In-Stock Today', 'drop')

view_or_drop_by_word(df, '#ArGameShowForMe', 'view')  #just spam 
df = view_or_drop_by_word(df, '#ArGameShowForMe', 'drop')

Data after dropping tweets containing word Merry Christmas :  4047
Data after dropping tweets containing word Happy new year :  4047
Data after dropping tweets containing word followers :  4047
Data after dropping tweets containing word contact me :  4047
Data after dropping tweets containing word Join :  4047
Data after dropping tweets containing word my channel :  4047
Data after dropping tweets containing word youtuber :  4047
Data after dropping tweets containing word See others view :  4047
Data after dropping tweets containing word How to solve :  4047
Data after dropping tweets containing word New Daily Update :  4047
Data after dropping tweets containing word NEW :  4047
Data after dropping tweets containing word How to Solve :  4047
Data after dropping tweets containing word #YoutubeChannel #Youtuber :  4047
Data after dropping tweets containing word In-Stock Today :  4047
3369  >>  #ArGameShowForMe #PlayStation ps4 + fifa18 today
3375  >>  #ArGameShowForMe #PlayStation #ArGam

In [ ]:
#save the post-filtered dataset
df.to_csv('playstation_2017-09-22_2018-02-02_tweets.csv', encoding='utf-8', index=False)
#check that everything is done correctly
df = pd.read_csv(r'playstation_2017-09-22_2018-02-02_tweets.csv')
df.shape

(4028, 7)

## Scraping **Xbox**

### **2020-2021**

In this section we extract data from 22-09-2020 to 01-02-2021, to capture opinions regarding **Xbox**, in particular for Xbox series s and x products. This window considers the period before and after that series was out (November 2020).

In [ ]:
## SETTING THE ADVANCED QUERY
ANDterms = ['xbox'] #all of these terms are mandatory
ORterms = ['xbox360', 'xbox1', 'xboxone', 'xboxonex', 'xboxones', 
           'xboxseries', 'xboxseriesx', 'xboxseriess'] #at least one of these terms should be included
NOTterms = ['follow', 'followme', 'check', 'giveaway', 'giveaways', 'page', 'twitch',
            'live', 'livestream', 'eBay', 'screenshot', 'youtube',
            'challenge', 'dm', 'shots', 'video', 'stream', 'Ad'] #none of these terms (probably add spam and irrelevant info)
hashtags = ['xbox'] #all these hashtags
lang = 'en'
date_until = '2021-02-02'
date_since = '2020-09-22'


## PREPARING THE FILE
filename = 'xbox_' + date_since + '_' + date_until + '_tweets.txt'
create_file(filename)

scrape_tweets(ANDterms, ORterms, NOTterms, hashtags, lang, date_until, date_since, filename)

In [ ]:
import pandas as pd
df = pd.read_csv(r'xbox_2020-09-22_2021-02-02_tweets.txt')
print(df.tail())

print("Data before dropping duplicate texts:\n ", df.count())
df.drop_duplicates(subset ="tweet_text",  keep = False, inplace = True)
print("Data after dropping duplicate texts:\n ", df.count())



               username                  datetime  ... retweets comments
36864         Boldsmack  2020-09-22T00:14:26.000Z  ...        2        0
36865     bryan tabares  2020-09-22T00:11:46.000Z  ...        0        0
36866  Chinmaya Nathany  2020-09-22T00:03:14.000Z  ...        0        1
36867            bjcro0  2020-09-22T00:02:54.000Z  ...        0        1
36868       playgames99  2020-09-22T00:02:00.000Z  ...        0        0

[5 rows x 7 columns]
Data before dropping duplicate texts:
  username          36784
datetime          36869
tweet_text        36869
retweeted_text     2126
likes             36869
retweets          36869
comments          36869
dtype: int64
Data after dropping duplicate texts:
  username          30083
datetime          30156
tweet_text        30156
retweeted_text     1916
likes             30156
retweets          30156
comments          30156
dtype: int64


**Post filter**

In [ ]:
view_or_drop_by_word(df, 'followers', 'view')
df = view_or_drop_by_word(df, 'followers', 'drop')

view_or_drop_by_word(df, 'happy new', 'view')
df = view_or_drop_by_word(df, 'happy new', 'drop')

view_or_drop_by_word(df, 'merry christmas', 'view')
df = view_or_drop_by_word(df, 'merry christmas', 'drop')

view_or_drop_by_word(df, 'Click', 'view') #just spam and ads
df = view_or_drop_by_word(df, 'Click', 'drop')

view_or_drop_by_word(df, 'Happy new year', 'view')
df = view_or_drop_by_word(df, 'Happy new year', 'drop')

view_or_drop_by_word(df, 'Merry Christmas', 'view')
df = view_or_drop_by_word(df, 'Merry Christmas', 'drop')


In [ ]:
#Save filtered dataset in the csv
df.to_csv('xbox_2020-09-22_2021-02-02_tweets.csv', encoding='utf-8', index=False)
df = pd.read_csv(r'xbox_2020-09-22_2021-02-02_tweets.csv')
df.head()

,username,datetime,tweet_text,retweeted_text,likes,retweets,comments
0,Desbee Jr.,2021-02-01T23:55:50.000Z,My first #darkmatterultra clip it was a good 5...,NaN,3,1,0
1,xboxnintendo222,2021-02-01T23:44:40.000Z,No photo mode! No cutscenes! In game bro! I Xb...,NaN,5,1,1
2,Germanicus,2021-02-01T23:37:14.000Z,"""Done"" #PS5 #XboxSeriesX #XboxSeries #Sony #Xbox",NaN,1,2,0
3,( FoAm ) XxDaddyxX,2021-02-01T23:27:01.000Z,* XBOX * Thursday Night FoAm 7:30 PM EST Battl...,NaN,0,0,0
4,David Ross,2021-02-01T23:26:02.000Z,Man those days were a nightmare. The WiFi adap...,"NeoNeo@NeoGameSparkFeb 1Ah man, I remember tho...",0,0,0


Chunking data to make preprocessing easier:

In [ ]:
GROUP_SIZE = 4000

def chunk(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

i = 0
for df_chunk in chunk(df, GROUP_SIZE):
  i += 1
  file_name = 'xbox_2020-09-22_2021-02-02_tweets' + str(i) + '.csv'
  print(file_name, "saved.")
  df_chunk.to_csv(file_name, encoding='utf-8', index=False)

xbox_2020-09-22_2021-02-02_tweets1.csv saved.
xbox_2020-09-22_2021-02-02_tweets2.csv saved.
xbox_2020-09-22_2021-02-02_tweets3.csv saved.
xbox_2020-09-22_2021-02-02_tweets4.csv saved.
xbox_2020-09-22_2021-02-02_tweets5.csv saved.
xbox_2020-09-22_2021-02-02_tweets6.csv saved.
xbox_2020-09-22_2021-02-02_tweets7.csv saved.
xbox_2020-09-22_2021-02-02_tweets8.csv saved.


### **2017-2018**

In this section we extract data from 22-09-2020 to 01-02-2021, to capture opinions regarding older **Xbox** consoles, in particular for Xbox 360 and xbox one products. This window considers the period before and after that Xbox One X was out (November 2017).

In [ ]:
## SETTING THE ADVANCED QUERY
ANDterms = ['xbox'] #all of these terms are mandatory
ORterms = ['xbox360', 'xbox1', 'xboxone', 'xboxonex', 'xboxones'] #at least one of these terms should be included
NOTterms = ['follow', 'followme', 'check', 'giveaway', 'giveaways', 'page', 'twitch',
            'live', 'livestream', 'eBay', 'screenshot', 'youtube',
            'challenge', 'dm', 'shots', 'video', 'stream', 'Ad', 'followers', 'follower'] #none of these terms (probably add spam and irrelevant info)
hashtags = ['xbox'] #all these hashtags
lang = 'en'
date_until = '2018-02-02'
date_since = '2017-09-22'


## PREPARING THE FILE
filename = 'xbox_' + date_since + '_' + date_until + '_tweets.txt'
create_file(filename)

scrape_tweets(ANDterms, ORterms, NOTterms, hashtags, lang, date_until, date_since, filename)

In [ ]:
import pandas as pd
df = pd.read_csv(r'xbox_2017-09-22_2018-02-02_tweets.txt')
print(df.tail())

print("Data before dropping duplicate texts:\n ", df.count())
df.drop_duplicates(subset ="tweet_text",  keep = False, inplace = True)
print("Data after dropping duplicate texts:\n ", df.count())



               username                  datetime  ... retweets comments
5500     Hangout Gaming  2017-09-22T13:11:27.000Z  ...        0        0
5501       The VR Realm  2017-09-22T11:54:44.000Z  ...        0        0
5502                Mac  2017-09-22T11:49:09.000Z  ...        0        0
5503              Jade   2017-09-22T11:08:34.000Z  ...        0        2
5504  Alessandro Biollo  2017-09-22T10:18:00.000Z  ...        0        0

[5 rows x 7 columns]
Data before dropping duplicate texts:
  username          5499
datetime          5505
tweet_text        5505
retweeted_text      93
likes             5505
retweets          5505
comments          5505
dtype: int64
Data after dropping duplicate texts:
  username          4683
datetime          4689
tweet_text        4689
retweeted_text      87
likes             4689
retweets          4689
comments          4689
dtype: int64


**Post filter**

In [ ]:
df = view_or_drop_by_word(df, 'followers', 'drop')

view_or_drop_by_word(df, 'happy new', 'view')
df = view_or_drop_by_word(df, 'happy new', 'drop')

view_or_drop_by_word(df, 'merry christmas', 'view')
df = view_or_drop_by_word(df, 'merry christmas', 'drop')

view_or_drop_by_word(df, 'Click', 'view') #just spam and ads
df = view_or_drop_by_word(df, 'Click', 'drop')

view_or_drop_by_word(df, 'Happy new year', 'view')
df = view_or_drop_by_word(df, 'Happy new year', 'drop')

view_or_drop_by_word(df, 'Merry Christmas', 'view')
df = view_or_drop_by_word(df, 'Merry Christmas', 'drop')

view_or_drop_by_word(df, 'sexy', 'view')
df = view_or_drop_by_word(df, 'sexy', 'drop')

view_or_drop_by_word(df, 'Merry', 'view')
df = view_or_drop_by_word(df, 'Merry', 'drop')

In [ ]:
#Store the post filtered data in csv
df.to_csv('xbox_2017-09-22_2018-02-02_tweets.csv', encoding='utf-8', index=False)